In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium as folium
import geopandas as gpd
import json
import os
import sys
import branca.colormap as cm

%load_ext lab_black

In [ ]:
pop_prov_df = pd.read_excel("./data/pobmun/pobmun22.xlsx", header=1)

In [ ]:
pop_prov_df

In [ ]:
provincias_geojson = "./data/georef-spain-provincia@public.geojson"

In [ ]:
with open("./data/georef-spain-provincia@public.geojson", "r") as f:
    province_data = json.load(f)

## Renaming so that Geojson and DF match names

In [ ]:
prov_names = []
for i in range(len(province_data["features"])):
    prov_names.append(province_data["features"][i]["properties"]["prov_name"])

In [ ]:
province_rename_dict = {
    "Alicante/Alacant": "Alacant",
    "Araba/Álava": "Araba",
    "Balears, Illes": "Illes Balears",
    "Castellón/Castelló": "Castelló",
    "Coruña, A": "A Coruña",
    "Palmas, Las": "Las Palmas",
    "Rioja, La": "La Rioja",
    "Valencia/València": "València",
}

In [ ]:
# Replace with my province dict
pop_prov_df["PROVINCIA"] = pop_prov_df["PROVINCIA"].replace(province_rename_dict)

In [ ]:
# Check if all provinces are in the geojson
for i in pop_prov_df["PROVINCIA"].unique():
    if i not in prov_names:
        print(i)

In [ ]:
male_pop = pop_prov_df.groupby("PROVINCIA")["HOMBRES"].sum()
female_pop = pop_prov_df.groupby("PROVINCIA")["MUJERES"].sum()

pop_df = pd.DataFrame({"men": male_pop, "women": female_pop})
pop_df = pop_df.sort_values(by="men", ascending=False)

In [ ]:
plt.figure(figsize=(10, 10))
pop_df.plot(kind="barh", figsize=(10, 10), width=0.8, cmap="tab20c")
plt.title("Population by province")
plt.xlabel("Population")
plt.ylabel("Province")
plt.show()

In [ ]:
pop_df["total"] = pop_df["men"] + pop_df["women"]

In [ ]:
pop_df.reset_index(inplace=True)

In [ ]:
# Add 'total' to GeoJSON properties
for feature in province_data["features"]:
    prov_name = feature["properties"]["prov_name"]
    if prov_name == "Territorio no asociado a ninguna provincia":
        # make it 0
        feature["properties"]["total"] = 0
        continue
    total_population = pop_df.loc[pop_df["PROVINCIA"] == prov_name, "total"].values[0]
    feature["properties"]["total"] = int(total_population)

In [ ]:
# cm.linear

In [ ]:
# Get min and max population
min_pop = pop_df["total"].min()
max_pop = pop_df["total"].max()

# Create a colormap
colormap = cm.linear.PuBuGn_09.scale(min_pop, max_pop)

In [ ]:
f = folium.Figure(width=1200, height=1000)

m = folium.Map(
    location=[40, -4],
    zoom_start=7,
    width=1200,
    height=1000,
    tiles="CartoDB positron",
    control_scale=True,
    no_touch=True,
).add_to(f)

folium.Choropleth(
    # geo_data=provincias_geojson,
    geo_data=province_data,
    name="choropleth",
    data=pop_df,
    columns=["PROVINCIA", "total"],
    key_on="feature.properties.prov_name",  # This is the key to match the geojson with the dataframe
    fill_color="PuBuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Population",
).add_to(m)


# add labels
style_function = lambda feature: {
    "fillColor": colormap(feature["properties"]["total"]),
    "color": "#000000",
    "fillOpacity": 0.5,
    "weight": 0.1,
}

highlight_function = lambda x: {
    "fillColor": "#000000",
    "color": "#000000",
    "fillOpacity": 0.50,
    "weight": 0.1,
}

tooltip = folium.features.GeoJsonTooltip(
    fields=["prov_name", "total"],
    aliases=["Province", "Population"],
    localize=True,
    sticky=True,
    labels=True,
    style="background-color: white;",
)

folium.GeoJson(
    province_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=tooltip,
    name="Provinces",
).add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
# Do it but with log scale

min_log_pop = pop_prov_df["log_total"].min()
max_log_pop = pop_prov_df["log_total"].max()

# Create a colormap
colormap = cm.linear.PuBuGn_09.scale(min_log_pop, max_log_pop)

In [ ]:
f = folium.Figure(width=1200, height=1000)

m = folium.Map(
    location=[40, -4],
    zoom_start=7,
    width=1200,
    height=1000,
    tiles="CartoDB positron",
    control_scale=True,
    no_touch=True,
).add_to(f)

folium.Choropleth(
    # geo_data=provincias_geojson,
    geo_data=province_data,
    name="choropleth",
    data=pop_df,
    columns=["PROVINCIA", "total"],
    key_on="feature.properties.prov_name",  # This is the key to match the geojson with the dataframe
    fill_color="PuBuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Population",
).add_to(m)


# add labels
style_function = lambda feature: {
    "fillColor": colormap(feature["properties"]["total"]),
    "color": "#000000",
    "fillOpacity": 0.5,
    "weight": 0.1,
}

highlight_function = lambda x: {
    "fillColor": "#000000",
    "color": "#000000",
    "fillOpacity": 0.50,
    "weight": 0.1,
}

tooltip = folium.features.GeoJsonTooltip(
    fields=["prov_name", "total"],
    aliases=["Province", "Population"],
    localize=True,
    sticky=True,
    labels=True,
    style="background-color: white;",
)

folium.GeoJson(
    province_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=tooltip,
    name="Provinces",
).add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
pop_df = pop_df.assign(m_w_ratio=pop_df["men"] / pop_df["women"])
pop_df.head()

In [ ]:
# Add 'm_w_ratio' to GeoJSON properties
for feature in province_data["features"]:
    prov_name = feature["properties"]["prov_name"]
    if prov_name == "Territorio no asociado a ninguna provincia":
        # make it 0
        feature["properties"]["m_w_ratio"] = 1
        continue
    m_w_ratio = pop_df.loc[pop_df["PROVINCIA"] == prov_name, "m_w_ratio"].values[0]
    feature["properties"]["m_w_ratio"] = float(m_w_ratio)

In [ ]:
# Get min and max population
min_ratio = pop_df["m_w_ratio"].min()
max_ratio = pop_df["m_w_ratio"].max()

# Create a colormap
colormap = cm.linear.PuBuGn_09.scale(min_ratio, max_ratio)
# colormap = cm.linear.YlGn_09.scale(min_pop, max_pop)

In [ ]:
f2 = folium.Figure(width=1200, height=1000)

m2 = folium.Map(
    location=[40, -4],
    zoom_start=7,
    width=1200,
    height=1000,
    tiles="CartoDB positron",
    control_scale=True,
    no_touch=True,
).add_to(f2)
folium.Choropleth(
    # geo_data=provincias_geojson,
    geo_data=province_data,
    name="choropleth",
    data=pop_df,
    columns=["PROVINCIA", "m_w_ratio"],
    key_on="feature.properties.prov_name",  # This is the key to match the geojson with the dataframe
    fill_color="PuBuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="men/women ratio",
).add_to(m2)

# add labels
# Create style function using the color function
style_function = lambda feature: {
    "fillColor": colormap(feature["properties"]["m_w_ratio"]),
    "color": "#000000",
    "fillOpacity": 0,
    "weight": 0.1,
}

highlight_function = lambda x: {
    "fillColor": "#000000",
    "color": "#000000",
    "fillOpacity": 0.5,
    "weight": 0.1,
}

tooltip = folium.features.GeoJsonTooltip(
    fields=["prov_name", "m_w_ratio"],
    aliases=["Province", "Men per Woman Ratio"],
    localize=True,
    sticky=True,
    labels=True,
    style="background-color: white;",
)

folium.GeoJson(
    province_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=tooltip,
    name="m_w_ratio",
).add_to(m2)

folium.LayerControl().add_to(m2)

In [ ]:
m2

In [ ]:
# Get min, max, and midpoint
min_ratio = pop_df["m_w_ratio"].min()
max_ratio = pop_df["m_w_ratio"].max()
mid_ratio = 1

# Create two colormaps
colormap1 = cm.linear.Blues_09.scale(min_ratio, mid_ratio)  # for values < 1
colormap2 = cm.linear.Greens_09.scale(mid_ratio, max_ratio)  # for values >= 1


# Function for getting colors
def get_color(feature):
    m_w_ratio = feature["properties"]["m_w_ratio"]
    if m_w_ratio < 1:
        return colormap1(m_w_ratio)
    else:
        return colormap2(m_w_ratio)

In [ ]:
f2 = folium.Figure(width=1200, height=1000)

m2 = folium.Map(
    location=[40, -4],
    zoom_start=7,
    width=1200,
    height=1000,
    tiles="CartoDB positron",
    control_scale=True,
    no_touch=True,
).add_to(f2)

# Get values below and above 1
pop_df_below_1 = pop_df[pop_df["m_w_ratio"] < 1]
pop_df_above_1 = pop_df[pop_df["m_w_ratio"] >= 1]

# Create two Choropleth maps
choropleth_below_1 = folium.Choropleth(
    geo_data=province_data,
    name="choropleth_below_1",
    data=pop_df_below_1,
    columns=["PROVINCIA", "m_w_ratio"],
    key_on="feature.properties.prov_name",
    fill_color="Blues",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="men/women ratio",
).add_to(m2)

choropleth_above_1 = folium.Choropleth(
    geo_data=province_data,
    name="choropleth_above_1",
    data=pop_df_above_1,
    columns=["PROVINCIA", "m_w_ratio"],
    key_on="feature.properties.prov_name",
    fill_color="Greens",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="men/women ratio above 1",
).add_to(m2)
# add labels
# Create style function using the color function
style_function = lambda feature: {
    "fillColor": get_color(feature),
    "color": "#000000",
    "fillOpacity": 0.7,
    "weight": 0.1,
}

highlight_function = lambda x: {
    "fillColor": "#000000",
    "color": "#000000",
    "fillOpacity": 0.5,
    "weight": 0.1,
}

tooltip = folium.features.GeoJsonTooltip(
    fields=["prov_name", "m_w_ratio"],
    aliases=["Province", "Men/Women Ratio"],
    localize=True,
    sticky=True,
    labels=True,
    style="background-color: white;",
)

folium.GeoJson(
    province_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=tooltip,
    name="m_w_ratio",
).add_to(m2)

folium.LayerControl().add_to(m2)

In [ ]:
m2

In [ ]:
# Provinces with most women:
print("Provinces with most women:")
print(
    pop_df[["PROVINCIA", "m_w_ratio"]]
    .sort_values(by="m_w_ratio")
    .head(10)
    .to_markdown()
)
pop_df[["PROVINCIA", "m_w_ratio"]].sort_values(by="m_w_ratio").head(10).plot(
    kind="bar", x="PROVINCIA", ylim=[0.9, 0.98], cmap="tab20c"
)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.title("Provinces with most men")
plt.xlabel("Province")
plt.ylabel("Man/Woman Ratio")
# no legend
plt.legend().remove()
plt.show()

In [ ]:
print("Provinces with most men:")
print(
    pop_df[["PROVINCIA", "m_w_ratio"]]
    .sort_values(by="m_w_ratio", ascending=False)
    .head(10)
    .to_markdown()
)
pop_df[["PROVINCIA", "m_w_ratio"]].sort_values(by="m_w_ratio", ascending=False).head(
    10
).plot(kind="bar", x="PROVINCIA", ylim=[0.98, 1.06], cmap="tab20c")
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.title("Provinces with most men")
plt.xlabel("Province")
plt.ylabel("Man/Woman Ratio")
# no legend
plt.legend().remove()
plt.show()

In [ ]:
# Plot a bar chart that is the nr of provinces with m_w_ratio larger than 1 vs smaller than 1
pop_df["m_w_ratio"].apply(lambda x: 1 if x > 1 else 0).value_counts().plot(
    kind="bar", x="m_w_ratio"
)
plt.title("Count of provinces with Man/Woman > 1 vs < 1")
plt.xticks([0, 1], ["More women", " More men"], rotation=0)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.xlabel("Amount of men per woman")
plt.ylabel("count")
plt.bar_label(plt.gca().containers[0], fmt="%d")
plt.show()

In [ ]:
# Bar plot of absolut values of men vs women
pop_df[["men", "women"]].sum().div(1000000).plot(kind="bar")
plt.title("Total population in Spain")
plt.xticks(rotation=0)
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.ylabel("Population (millions)")
plt.bar_label(ax.containers[0], fmt="%.1f")
plt.show()

## Municipalities

In [ ]:
pop_prov_df

In [ ]:
pop_prov_df = pop_prov_df.assign(total=pop_prov_df["HOMBRES"] + pop_prov_df["MUJERES"])

In [ ]:
with open("./data/georef-spain-municipio@public.geojson", "r") as f:
    municipality_data = json.load(f)

In [ ]:
mun_names = []
for i in range(len(municipality_data["features"])):
    mun_names.append(municipality_data["features"][i]["properties"]["mun_name"])

mun_codes = []
for i in range(len(municipality_data["features"])):
    mun_codes.append(municipality_data["features"][i]["properties"]["mun_code"])

In [ ]:
# Create a new column called mun_code which is the CPRO + CMUN as strings. CPRO should be 2 digits and CMUN should be 3 digits
pop_prov_df["mun_code"] = pop_prov_df["CPRO"].astype(str).str.zfill(2) + pop_prov_df[
    "CMUN"
].astype(str).str.zfill(3)

In [ ]:
missing_mun_codes = []

for code in mun_codes:
    if code not in pop_prov_df["mun_code"].unique():
        missing_mun_codes.append(code)

# Remove the not matching municipalities from the municipality data
municipality_data["features"] = [
    feature
    for feature in municipality_data["features"]
    if feature["properties"]["mun_code"] not in missing_mun_codes
]

In [ ]:
# Use logarithmic scale for the total population to enhance the differences
pop_prov_df["log_total"] = np.log(pop_prov_df["total"])

In [ ]:
# Add 'total' and "log_total" to GeoJSON properties
for feature in municipality_data["features"]:
    mun_code = feature["properties"]["mun_code"]
    try:
        total_population = pop_prov_df.loc[
            pop_prov_df["mun_code"] == mun_code, "total"
        ].values[0]
        log_total_population = pop_prov_df.loc[
            pop_prov_df["mun_code"] == mun_code, "log_total"
        ].values[0]

        feature["properties"]["total"] = int(total_population)
        feature["properties"]["log_total"] = float(log_total_population)
    except:
        print(f"error on {mun_code}")

## Plot total pop by municipality

In [ ]:
# Get min and max population use logarithmic for skewness

min_log_pop = pop_prov_df["log_total"].min()
max_log_pop = pop_prov_df["log_total"].max()

# Create a colormap
colormap = cm.linear.PuBuGn_09.scale(min_log_pop, max_log_pop)

In [ ]:
f = folium.Figure(width=1200, height=1000)

m = folium.Map(
    location=[40, -4],
    zoom_start=7,
    width=1200,
    height=1000,
    tiles="CartoDB positron",
    control_scale=True,
    no_touch=True,
).add_to(f)

folium.Choropleth(
    geo_data=municipality_data,
    name="choropleth",
    data=pop_prov_df,
    columns=["mun_code", "log_total"],  # use the log scale column here
    key_on="feature.properties.mun_code",
    fill_color="PuBuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Log Population",  # note the legend name change
).add_to(m)


# add labels
style_function = lambda feature: {
    "fillColor": colormap(np.log(feature["properties"]["total"])),  # apply log here too
    "color": "#000000",
    "fillOpacity": 0.5,
    "weight": 0.1,
}

highlight_function = lambda x: {
    "fillColor": "#000000",
    "color": "#000000",
    "fillOpacity": 0.50,
    "weight": 0.1,
}

tooltip = folium.features.GeoJsonTooltip(
    fields=["mun_name", "log_total"],  # and here
    aliases=["Municipality", "Log Population"],  # and here
    localize=True,
    sticky=True,
    labels=True,
    style="background-color: white;",
)


folium.GeoJson(
    municipality_data,
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=tooltip,
    name="Municipalities",
).add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
# m